In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

def detect_colors(image, k=4):
    """Detects k dominant colors in the image using K-means clustering."""
    reshaped_image = image.reshape((-1, 3))
    reshaped_image = reshaped_image[np.any(reshaped_image > 20, axis=1)]  # Remove black pixels

    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(reshaped_image)
    dominant_colors = kmeans.cluster_centers_.astype(int)

    return dominant_colors

def extract_trajectories(image, dominant_colors):
    """Extracts trajectory points based on dynamically detected colors."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    trajectory_points = {}

    for i, color in enumerate(dominant_colors):
        lower_bound = np.array([max(0, c - 30) for c in color], dtype=np.uint8)
        upper_bound = np.array([min(255, c + 30) for c in color], dtype=np.uint8)

        mask = cv2.inRange(hsv, lower_bound, upper_bound)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        points = []
        for contour in contours:
            for point in contour:
                points.append(point[0])

        if points:
            trajectory_points[f"color_{i}"] = np.array(points)

    return trajectory_points

def compute_noise_with_polyfit(points, degree=3):
    """Fits a polynomial regression model and computes deviation of actual points."""
    points = points[np.argsort(points[:, 0])]
    x, y = points[:, 0].reshape(-1, 1), points[:, 1]

    unique_x, indices = np.unique(x, return_index=True)
    y = y[indices]

    if len(unique_x) < degree + 1:
        return np.nan, np.nan, np.full_like(y, np.nan), np.full_like(y, np.nan)

    poly = PolynomialFeatures(degree)
    x_poly = poly.fit_transform(unique_x.reshape(-1, 1))
    model = LinearRegression()
    model.fit(x_poly, y)

    y_pred = model.predict(x_poly)
    noise_levels = y - y_pred  # Signed deviation

    return noise_levels.mean(), noise_levels.std(), noise_levels, y_pred

def analyze_image_noise(image_path):
    """Processes a single image to compute detailed noise statistics."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image {image_path}")
        return None

    dominant_colors = detect_colors(image)
    trajectories = extract_trajectories(image, dominant_colors)

    all_noise = []
    noise_results = {}

    for color, points in trajectories.items():
        mean_noise, std_noise, noise_values, reference_y = compute_noise_with_polyfit(points)
        noise_results[color] = {
            "mean_noise": mean_noise,
            "std_noise": std_noise,
            "num_points": len(points)
        }

        if isinstance(noise_values, np.ndarray):  # Ensure it's an array before extending
            all_noise.extend(noise_values.tolist())

    if not all_noise:
        return None  # No valid noise extracted

    # Compute noise histogram
    noise_hist, bin_edges = np.histogram(all_noise, bins=20, density=True)
    
    noise_summary = {
        "mean_noise": np.mean(all_noise),
        "std_noise": np.std(all_noise),
        "histogram": noise_hist.tolist(),
        "bin_edges": bin_edges.tolist(),
    }

    return noise_summary

def process_folder(folder_path, output_file="noise_profile.json"):
    """Processes all images in a folder and saves detailed noise analysis."""
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    noise_data = {}

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        noise_summary = analyze_image_noise(image_path)

        if noise_summary:
            noise_data[image_file] = noise_summary
            print(f"Processed {image_file}: Mean Noise {noise_summary['mean_noise']:.2f}, Std {noise_summary['std_noise']:.2f}")

    with open(output_file, "w") as f:
        json.dump(noise_data, f, indent=4)

    print(noise_data)

# Usage
folder_path = r"C:\Users\tnlab\OneDrive\Desktop\Noise Plots Ten"  # Change this to your actual folder path
process_folder(folder_path)


Processed Apr28-2015-Xzhao.png: Mean Noise -0.00, Std 191.63
Processed April03-2017-EvanV.png: Mean Noise 0.00, Std 381.23


c:\ProgramData\anaconda3\Lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


Processed April03-2017-KunX.png: Mean Noise -2556528.15, Std 74051107.80
Processed April03-2017-LandonP.png: Mean Noise 0.00, Std 304.37
Processed April03-2017-ShelbyG.png: Mean Noise 0.00, Std 225.14


c:\ProgramData\anaconda3\Lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


Processed April04-2017-JustinK.png: Mean Noise -1532822.02, Std 57352948.28
Processed April05-2017-BrittanyL.png: Mean Noise 0.00, Std 294.48
Processed April05-2017-ChrisW.png: Mean Noise -0.00, Std 240.02
Processed April05-2017-HunterW.png: Mean Noise 0.00, Std 347.92
Processed April05-2017-KaitlynB.png: Mean Noise 0.00, Std 279.94
Processed April05-2017-NathanO.png: Mean Noise 0.00, Std 206.35
Processed April06-2017-AudreyL.png: Mean Noise -0.00, Std 110.36
Processed April06-2017-HamptonC.png: Mean Noise 0.00, Std 212.81
Processed April06-2017-MattK.png: Mean Noise 0.00, Std 296.41


...

Processed Oct27-2016-MichaelH.png: Mean Noise -1064164.34, Std 47792709.52
Processed Oct31-2016-BolorzulD.png: Mean Noise -0.00, Std 311.29


c:\ProgramData\anaconda3\Lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


Processed Sep08-2016-WilliamL.png: Mean Noise -2024018.52, Std 65897268.69
Processed Sept26-2016-MattL.png: Mean Noise -0.00, Std 161.52


c:\ProgramData\anaconda3\Lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


Processed Sept27-2016-OmeedN.png: Mean Noise -4917901.48, Std 102649536.11
Processed Sept28-2016-JoshK.png: Mean Noise 0.00, Std 174.65
Processed Sept30-2016-DineshV.png: Mean Noise 0.00, Std 214.77
{'Apr28-2015-Xzhao.png': {'mean_noise': -1.8309779496447207e-13, 'std_noise': 191.62821524648191, 'histogram': [0.0014922197149333663, 0.000818102913888906, 0.0005497651581333455, 0.0004385031618444539, 0.0018325505271111504, 0.0015118541848667, 0.0026048396778222765, 0.004659914197511214, 0.0019765366399555995, 0.0014922197149333644, 0.0015380334781111451, 0.00041886869191112033, 0.0004123238686000086, 0.00021597916926667117, 0.00026179293244445054, 0.00032069634224445113, 0.00024215846251111615, 0.00018979987602222665, 0.0004908617483333436, 0.00037959975204445237], 'bin_edges': [-347.5214248101986, -301.7477540879733, -255.97408336574796, -210.20041264352267, -164.42674192129735, -118.65307119907203, -72.87940047684674, -27.105729754621393, 18.667940967603897, 64.44161168982919, 110.2152